In [1]:
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import math
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset,random_split

In [2]:
#import os
#os.chdir("./../.")
#os.getcwd()

In [3]:
torch.autograd.set_detect_anomaly(True)

### Filtering the output 2

In [4]:
torch.__version__
torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = "cpu"
print(device)

cuda:0


In [5]:
with open("data/surrogate_model/animation_label.pkl", "rb") as f:
    surrogate2 = pickle.load(f)

#Filter with only good or very good ratings
#filtered_output = surrogate[surrogate)][['label'].isin(['Good','Very Good']["file","animation_id","model_output","label"]]
filtered_output2 = surrogate2[["file","animation_id","model_output","label"]]

# dictionary for mapping

mapping_dict = {"Very Good": 6, "Good": 5, "Bad": 4,"Okay":3, "Very Bad": 2, "no_rating": 1}

# Create another column changing the label into ints

filtered_output2['label2'] = filtered_output2['label'].replace(mapping_dict)


# get the names of unique logos by splitting with animation number
logos = filtered_output2["file"].str.split("_animation").str[0].unique()

#print(logos)

# create a data frame for the collected best animations
bestoutput2 = pd.DataFrame()

# go through each logo to find the best animation
for logo in logos:

    # make a data frame that contains all the animations of one logo
    temp = filtered_output2[filtered_output2["file"].str.contains(logo)]

    #display(temp)

    # create a sum 
    mean_by_label = temp.groupby('file')['label2'].mean().reset_index()

    #print(mean_by_label)

    bestlogo = mean_by_label.loc[mean_by_label['label2'].idxmax()]

    #print(bestlogo)

    # get all the animated paths with the best animation of the logo
    best_animations2 = temp[temp["file"]==bestlogo["file"]]

    # add to the file
    bestoutput2 = pd.concat([bestoutput2,best_animations2],axis=0, ignore_index=True)
bestoutput2 = bestoutput2.sort_values(by=['file','animation_id'])[["file", "animation_id", "model_output"]]
display(bestoutput2)


maxLen = bestoutput2['file'].value_counts().iloc[0]

filenames = bestoutput2["file"].unique()
print(len(filenames))
list = []


for name in filenames:
    seq = bestoutput2[bestoutput2["file"]==name]

    seq = pd.DataFrame(seq["model_output"])
    
    sos = pd.DataFrame({"model_output" : [np.repeat(30, 12)]})
    
    eos = pd.DataFrame({"model_output" : [np.repeat(50, 12)]})
    #print(seq.shape, sos.shape, eos.shape)
    seq = pd.concat([sos, seq, eos], ignore_index=True)
    seq['model_output'] = seq['model_output'].apply(lambda arr: arr.tolist())
    while len(seq) < maxLen+2:
        seq = pd.concat([seq, pd.DataFrame({"model_output" : [np.repeat(-10, 12)]})], ignore_index=True)
    
  
    tensors = []
    for value in seq.values:
        #print(value, type(value), value.dtype)
        tensors.append(torch.tensor(value[0]))
    #print(torch.stack(tensors))
    list.append(torch.stack(tensors))

outTensor2 = torch.stack(list)

zeros_to_add = torch.zeros(outTensor2.shape[0], outTensor2.shape[1], 244)
outTensor2 = torch.cat([outTensor2, zeros_to_add], dim=2)

outTensor2 = outTensor2.to(device)
print(outTensor2.max())
print(outTensor2.shape)
print(outTensor2)

C:\Users\okan2\AppData\Local\Temp\ipykernel_25320\3746254375.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_output2['label2'] = filtered_output2['label'].replace(mapping_dict)


,file,animation_id,model_output
436,logo_0_animation_0,0,"[0, 0, 0, 1, 0, 0, -1.0, -1.0, -1.0, -1.0, 0.8..."
439,logo_0_animation_0,1,"[0, 0, 0, 0, 1, 0, -1.0, -1.0, -1.0, -1.0, -1...."
438,logo_0_animation_0,3,"[0, 0, 0, 0, 0, 1, -1.0, -1.0, -1.0, -1.0, -1...."
437,logo_0_animation_0,4,"[0, 0, 1, 0, 0, 0, -1.0, -1.0, -1.0, 0.4205715..."
435,logo_0_animation_0,5,"[0, 0, 0, 0, 1, 0, -1.0, -1.0, -1.0, -1.0, -1...."
...,...,...,...
892,logo_99_animation_0,21,"[0, 0, 0, 1, 0, 0, -1.0, -1.0, -1.0, -1.0, 0.4..."
811,logo_99_animation_0,22,"[0, 0, 0, 1, 0, 0, -1.0, -1.0, -1.0, -1.0, 0.8..."
891,logo_99_animation_0,22,"[0, 0, 0, 1, 0, 0, -1.0, -1.0, -1.0, -1.0, 0.8..."
810,logo_99_animation_0,23,"[0, 0, 0, 0, 1, 0, -1.0, -1.0, -1.0, -1.0, -1...."


359
tensor(50., device='cuda:0')
torch.Size([359, 14, 256])
tensor([[[ 30.,  30.,  30.,  ...,   0.,   0.,   0.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         ...,
         [-10., -10., -10.,  ...,   0.,   0.,   0.],
         [-10., -10., -10.,  ...,   0.,   0.,   0.],
         [-10., -10., -10.,  ...,   0.,   0.,   0.]],

        [[ 30.,  30.,  30.,  ...,   0.,   0.,   0.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [ 50.,  50.,  50.,  ...,   0.,   0.,   0.],
         ...,
         [-10., -10., -10.,  ...,   0.,   0.,   0.],
         [-10., -10., -10.,  ...,   0.,   0.,   0.],
         [-10., -10., -10.,  ...,   0.,   0.,   0.]],

        [[ 30.,  30.,  30.,  ...,   0.,   0.,   0.],
         [  1.,   0.,   0.,  ...,   0.,   0.,   0.],
         [  0.,   0.,   1.,  ...,   0.,   0.,   0.],
         ...,
         [-10., -10., -10.,  ...,   0.,   0.,   0.],
         [-10., -10., -10.,  ...,   0.,   0., 

### Creating the input tensor with the diltered output

In [6]:
with open("data\embeddings\path_embedding.pkl", "rb") as f:
    inp = pickle.load(f)

bestoutput2["filename"] = bestoutput2["file"].str.split("_animation").str[0]

display(bestoutput2)

inp['animation_id'] = inp['animation_id'].astype(int)

#names = bestoutput2["file"].str.replace("_animation_0", "")

#input = inp[(inp["filename"].isin(bestoutput2["filename"])) & (inp["animation_id"].isin(bestoutput2["animation_id"]))]
input = pd.merge(bestoutput2, inp, on=['filename', 'animation_id'], how='inner')
input = input.drop(['model_output', 'file'], axis=1)

input = input.sort_values(by=['filename','animation_id'])
display(input)
maxLen = bestoutput2['file'].value_counts().iloc[0]
filenames = input["filename"].unique()
#print(filenames)
list = []
for name in filenames:
    #print(name)
    seq = input[input["filename"]==name].loc[:, ~inp.columns.isin(["filename","animation_id"])]
    #print(seq)
    #seq = pd.concat([seq, pd.DataFrame(0, index=seq.index, columns=range(256, 268))], axis=1, ignore_index=True)

    sos = pd.DataFrame([[30]*256])
    
    eos = pd.DataFrame([[50]*256])

    seq = pd.concat([sos, seq, eos], ignore_index=True)

    while len(seq) < maxLen +2:
        seq = pd.concat([seq, pd.DataFrame([[-10]*256])], ignore_index=True)

    list.append(torch.tensor(seq.values))
    #print(list)
inpTensor2 = torch.stack(list)
inpTensor2 = inpTensor2.to(device)
print(inpTensor2)
print(inpTensor2.shape)

,file,animation_id,model_output,filename
436,logo_0_animation_0,0,"[0, 0, 0, 1, 0, 0, -1.0, -1.0, -1.0, -1.0, 0.8...",logo_0
439,logo_0_animation_0,1,"[0, 0, 0, 0, 1, 0, -1.0, -1.0, -1.0, -1.0, -1....",logo_0
438,logo_0_animation_0,3,"[0, 0, 0, 0, 0, 1, -1.0, -1.0, -1.0, -1.0, -1....",logo_0
437,logo_0_animation_0,4,"[0, 0, 1, 0, 0, 0, -1.0, -1.0, -1.0, 0.4205715...",logo_0
435,logo_0_animation_0,5,"[0, 0, 0, 0, 1, 0, -1.0, -1.0, -1.0, -1.0, -1....",logo_0
...,...,...,...,...
892,logo_99_animation_0,21,"[0, 0, 0, 1, 0, 0, -1.0, -1.0, -1.0, -1.0, 0.4...",logo_99
811,logo_99_animation_0,22,"[0, 0, 0, 1, 0, 0, -1.0, -1.0, -1.0, -1.0, 0.8...",logo_99
891,logo_99_animation_0,22,"[0, 0, 0, 1, 0, 0, -1.0, -1.0, -1.0, -1.0, 0.8...",logo_99
810,logo_99_animation_0,23,"[0, 0, 0, 0, 1, 0, -1.0, -1.0, -1.0, -1.0, -1....",logo_99


,animation_id,filename,0,1,2,3,4,5,6,7,...,246,247,248,249,250,251,252,253,254,255
0,0,logo_0,0.763518,-0.982797,-0.446681,1.089468,-0.070563,0.710206,-0.491675,-1.631172,...,0.339061,0.022934,0.195161,-0.046488,-0.492103,-0.605836,-1.282879,0.613195,0.297194,-0.172312
1,1,logo_0,0.851117,-1.775123,0.649689,-0.688600,0.216071,0.135211,-1.748761,-1.347670,...,-1.616431,-1.572003,0.242460,0.430259,0.079752,-1.039526,-0.696104,0.090277,-0.228757,0.144372
2,3,logo_0,0.291136,-0.928242,0.265542,-0.261439,-0.386160,1.256256,-0.414706,-1.206105,...,-0.446020,-1.369758,0.356421,1.456656,0.468766,-1.077724,-0.548627,-0.300660,0.632805,-0.136473
3,4,logo_0,0.504446,-0.543099,0.915062,1.293575,-0.849605,1.120387,-0.637641,-1.337280,...,0.162753,0.206993,-0.201259,-1.087391,-0.597388,-0.992079,-0.851486,-0.225463,-0.549269,0.088637
4,5,logo_0,-0.641569,-0.657125,-0.105109,-0.031630,-0.572032,0.912017,-0.569627,-1.573482,...,-1.079276,0.285482,1.532865,-0.375210,-0.249130,-0.551393,-1.024246,0.623726,-1.073305,0.166613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902,21,logo_99,0.729620,-1.035586,0.390709,1.910684,-0.489203,2.197111,0.424709,-0.389426,...,1.130544,-1.892641,-0.690288,1.371062,0.718980,-0.827454,-0.557172,0.235162,0.523978,-0.592003
903,22,logo_99,1.147938,-0.799806,0.664411,1.610482,-0.612273,1.194499,0.076532,-0.662109,...,0.415811,-1.569571,-0.424470,0.518902,0.160598,-1.283986,-0.463894,0.319661,0.785325,-0.294870
904,22,logo_99,1.147938,-0.799806,0.664411,1.610482,-0.612273,1.194499,0.076532,-0.662109,...,0.415811,-1.569571,-0.424470,0.518902,0.160598,-1.283986,-0.463894,0.319661,0.785325,-0.294870
905,23,logo_99,1.645128,-1.911424,0.806118,0.850210,0.554424,1.578238,-0.722733,-0.704658,...,0.883759,-2.000007,-0.517095,2.213172,0.733567,-0.577074,-0.710312,-0.591997,0.836300,-0.720247


tensor([[[ 3.0000e+01,  3.0000e+01,  3.0000e+01,  ...,  3.0000e+01,
           3.0000e+01,  3.0000e+01],
         [ 7.6352e-01, -9.8280e-01, -4.4668e-01,  ...,  6.1320e-01,
           2.9719e-01, -1.7231e-01],
         [ 8.5112e-01, -1.7751e+00,  6.4969e-01,  ...,  9.0277e-02,
          -2.2876e-01,  1.4437e-01],
         ...,
         [-1.0000e+01, -1.0000e+01, -1.0000e+01,  ..., -1.0000e+01,
          -1.0000e+01, -1.0000e+01],
         [-1.0000e+01, -1.0000e+01, -1.0000e+01,  ..., -1.0000e+01,
          -1.0000e+01, -1.0000e+01],
         [-1.0000e+01, -1.0000e+01, -1.0000e+01,  ..., -1.0000e+01,
          -1.0000e+01, -1.0000e+01]],

        [[ 3.0000e+01,  3.0000e+01,  3.0000e+01,  ...,  3.0000e+01,
           3.0000e+01,  3.0000e+01],
         [ 9.5908e-01,  4.6884e-01,  3.8706e-01,  ...,  1.7704e-02,
           4.0159e-01,  1.7431e+00],
         [ 5.0000e+01,  5.0000e+01,  5.0000e+01,  ...,  5.0000e+01,
           5.0000e+01,  5.0000e+01],
         ...,
         [-1.0000e+01, -1

In [7]:
test = inpTensor2[:1,:,:]
test = test.to(torch.float32)
test = test.to(device)
print(test, test.shape)

tensor([[[ 30.0000,  30.0000,  30.0000,  ...,  30.0000,  30.0000,  30.0000],
         [  0.7635,  -0.9828,  -0.4467,  ...,   0.6132,   0.2972,  -0.1723],
         [  0.8511,  -1.7751,   0.6497,  ...,   0.0903,  -0.2288,   0.1444],
         ...,
         [-10.0000, -10.0000, -10.0000,  ..., -10.0000, -10.0000, -10.0000],
         [-10.0000, -10.0000, -10.0000,  ..., -10.0000, -10.0000, -10.0000],
         [-10.0000, -10.0000, -10.0000,  ..., -10.0000, -10.0000, -10.0000]]],
       device='cuda:0') torch.Size([1, 14, 256])


### Model

In [8]:
print(inpTensor2, inpTensor2.shape)

print(outTensor2, outTensor2.shape)

tensor([[[ 3.0000e+01,  3.0000e+01,  3.0000e+01,  ...,  3.0000e+01,
           3.0000e+01,  3.0000e+01],
         [ 7.6352e-01, -9.8280e-01, -4.4668e-01,  ...,  6.1320e-01,
           2.9719e-01, -1.7231e-01],
         [ 8.5112e-01, -1.7751e+00,  6.4969e-01,  ...,  9.0277e-02,
          -2.2876e-01,  1.4437e-01],
         ...,
         [-1.0000e+01, -1.0000e+01, -1.0000e+01,  ..., -1.0000e+01,
          -1.0000e+01, -1.0000e+01],
         [-1.0000e+01, -1.0000e+01, -1.0000e+01,  ..., -1.0000e+01,
          -1.0000e+01, -1.0000e+01],
         [-1.0000e+01, -1.0000e+01, -1.0000e+01,  ..., -1.0000e+01,
          -1.0000e+01, -1.0000e+01]],

        [[ 3.0000e+01,  3.0000e+01,  3.0000e+01,  ...,  3.0000e+01,
           3.0000e+01,  3.0000e+01],
         [ 9.5908e-01,  4.6884e-01,  3.8706e-01,  ...,  1.7704e-02,
           4.0159e-01,  1.7431e+00],
         [ 5.0000e+01,  5.0000e+01,  5.0000e+01,  ...,  5.0000e+01,
           5.0000e+01,  5.0000e+01],
         ...,
         [-1.0000e+01, -1

In [9]:
class PositionalEncoding(nn.Module):
    def __init__(self, dim_model, dropout_p, max_len):
        super().__init__()
        # Modified version from: https://pytorch.org/tutorials/beginner/transformer_tutorial.html
        # max_len determines how far the position can have an effect on a token (window)
        
        # Info
        self.dropout = nn.Dropout(dropout_p)
        
        # Encoding - From formula
        pos_encoding = torch.zeros(max_len, dim_model)
        positions_list = torch.arange(0, max_len, dtype=torch.float).view(-1, 1) # 0, 1, 2, 3, 4, 5
        division_term = torch.exp(torch.arange(0, dim_model, 2).float() * (-math.log(10000.0)) / dim_model) # 1000^(2i/dim_model)
        
        # PE(pos, 2i) = sin(pos/1000^(2i/dim_model))
        pos_encoding[:, 0::2] = torch.sin(positions_list * division_term)
        
        # PE(pos, 2i + 1) = cos(pos/1000^(2i/dim_model))
        pos_encoding[:, 1::2] = torch.cos(positions_list * division_term)
        
        # Saving buffer (same as parameter without gradients needed)
        pos_encoding = pos_encoding.unsqueeze(0).transpose(0, 1)
        self.register_buffer("pos_encoding",pos_encoding)
        
    def forward(self, token_embedding: torch.tensor) -> torch.tensor:
        # Residual connection + pos encoding
        return self.dropout(token_embedding + self.pos_encoding[:token_embedding.size(0), :])

In [67]:

class Transformer(nn.Module):
    """
    Model from "A detailed guide to Pytorch's nn.Transformer() module.", by
    Daniel Melchor: https://medium.com/@danielmelchor/a-detailed-guide-to-pytorchs-nn-transformer-module-c80afbc9ffb1
    """
    # Constructor
    def __init__(
        self,
        num_tokens,
        dim_model,
        num_heads,
        num_encoder_layers,
        num_decoder_layers,
        dropout_p,
    ):
        super().__init__()

        # INFO
        self.model_type = "Transformer"
        self.dim_model = dim_model

        # LAYERS
        self.positional_encoder = PositionalEncoding(
            dim_model=dim_model, dropout_p=dropout_p, max_len=5000
        )
        self.embedding = nn.Embedding(num_tokens, dim_model)
        self.transformer = nn.Transformer(
            d_model=dim_model,
            nhead=num_heads,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dropout=dropout_p,
            batch_first=True
        )
        self.out = nn.Linear(dim_model, num_tokens)
        
    def forward(self, src, tgt, tgt_mask=None, src_pad_mask=None, tgt_pad_mask=None, batch_first=True):
        # Src size must be (batch_size, src sequence length)
        # Tgt size must be (batch_size, tgt sequence length)

        # Embedding + positional encoding - Out size = (batch_size, sequence length, dim_model)
        #src = self.embedding(src) * math.sqrt(self.dim_model)
        #tgt = self.embedding(tgt) * math.sqrt(self.dim_model)
        src = self.positional_encoder(src)
        tgt = self.positional_encoder(tgt)
        

        # Transformer blocks - Out size = (sequence length, batch_size, num_tokens)
        #print(src.shape, tgt.shape)
        transformer_out = self.transformer(src, tgt, tgt_mask=tgt_mask, src_key_padding_mask=src_pad_mask, tgt_key_padding_mask=tgt_pad_mask)
        out = self.out(transformer_out)
        
        return out
      
    def get_tgt_mask(self, size) -> torch.tensor:
        # Generates a squeare matrix where the each row allows one word more to be seen
        mask = torch.tril(torch.ones(size, size) == 1) # Lower triangular matrix
        mask = mask.float()
        mask = mask.masked_fill(mask == 0, float('-inf')) # Convert zeros to -inf
        mask = mask.masked_fill(mask == 1, float(0.0)) # Convert ones to 0
        
        # EX for size=5:
        # [[0., -inf, -inf, -inf, -inf],
        #  [0.,   0., -inf, -inf, -inf],
        #  [0.,   0.,   0., -inf, -inf],
        #  [0.,   0.,   0.,   0., -inf],
        #  [0.,   0.,   0.,   0.,   0.]]
        
        return mask
    
    def create_pad_mask(self, matrix: torch.tensor, pad_token: int) -> torch.tensor:
        # If matrix = [1,2,3,0,0,0] where pad_token=0, the result mask is
        # [False, False, False, True, True, True]
        mask = []
        #print(matrix)
        for i in range(0, matrix.size(0)):
            seq = []
            for j in range(0, matrix.size(1)):
                if matrix[i,j,0] == pad_token:
                    seq.append(True)
                else:
                    seq.append(False)
            mask.append(seq)
        result = torch.tensor(mask)
        #print(matrix, result, result.shape)
        return result

In [63]:
model = Transformer(
    num_tokens=256, dim_model=256, num_heads=4, num_encoder_layers=8, num_decoder_layers=8, dropout_p=0.1
).to(device)
opt = torch.optim.SGD(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

In [64]:

def train_loop(model, opt, loss_fn, dataloader):
    """
    Method from "A detailed guide to Pytorch's nn.Transformer() module.", by
    Daniel Melchor: https://medium.com/@danielmelchor/a-detailed-guide-to-pytorchs-nn-transformer-module-c80afbc9ffb1
    """
    
    model.train()
    total_loss = 0
    
    for batch in dataloader:
        X, y = batch[0], batch[1]
        X, y = torch.tensor(X).to(device), torch.tensor(y).to(device)

        #print(X.shape, y.shape)
        # Now we shift the tgt by one so with the <SOS> we predict the token at pos 1
        y_input = y[:,:-1]
        y_expected = y[:,1:]

        #print(y.shape, y_input, y_input.shape, y_expected, y_expected.shape)
        
        # Get mask to mask out the next words
        sequence_length = y.size(1)
        tgt_mask = model.get_tgt_mask(sequence_length).to(device)
        pad_mask_src = model.create_pad_mask(X, -10).to(device)
        pad_mask_tgt = model.create_pad_mask(y, -10).to(device)

        # Standard training except we pass in y_input and tgt_mask
        #print(X.shape, y.shape)
        pred = model(X, y, tgt_mask, src_pad_mask=pad_mask_src, tgt_pad_mask=pad_mask_tgt)

        # Permute pred to have batch size first again
        #pred = pred.permute(1, 2, 0)      
        loss = loss_fn(pred, y)

        opt.zero_grad()
        loss.backward()
        opt.step()
    
        total_loss += loss.detach().item()
        
    return total_loss / len(dataloader)

In [65]:
def validation_loop(model, loss_fn, dataloader):
    """
    Method from "A detailed guide to Pytorch's nn.Transformer() module.", by
    Daniel Melchor: https://medium.com/@danielmelchor/a-detailed-guide-to-pytorchs-nn-transformer-module-c80afbc9ffb1
    """
    
    model.eval()
    total_loss = 0
    
    with torch.no_grad():
        for batch in dataloader:
            X, y = batch[0], batch[1]
            X, y = torch.tensor(X, device=device), torch.tensor(y, device=device)

            # Now we shift the tgt by one so with the <SOS> we predict the token at pos 1
            y_input = y[:,:-1]
            y_expected = y[:,1:]
            
            # Get mask to mask out the next words
            sequence_length = y.size(1)
            tgt_mask = model.get_tgt_mask(sequence_length).to(device)
            pad_mask_src = model.create_pad_mask(X, -10)
            pad_mask_tgt = model.create_pad_mask(y, -10)

            # Standard training except we pass in y_input and src_mask
            #print("val ", X.shape, y.shape, X.dtype, y.dtype)
            pred = model(X, y, tgt_mask)

            # Permute pred to have batch size first again
            #pred = pred.permute(1, 2, 0)      
            loss = loss_fn(pred, y)
            total_loss += loss.detach().item()
        
    return total_loss / len(dataloader)

In [66]:
train_dataloader = DataLoader(TensorDataset(inpTensor2.float(), outTensor2.float()), batch_size=50, shuffle=True, drop_last=True)
val_dataloader = DataLoader(TensorDataset(inpTensor2.float(), outTensor2.float()),batch_size=50, shuffle=True,  drop_last=True)

def fit(model, opt, loss_fn, train_dataloader, val_dataloader, epochs):
    """
    Method from "A detailed guide to Pytorch's nn.Transformer() module.", by
    Daniel Melchor: https://medium.com/@danielmelchor/a-detailed-guide-to-pytorchs-nn-transformer-module-c80afbc9ffb1
    """
    
    # Used for plotting later on
    train_loss_list, validation_loss_list = [], []
    
    print("Training and validating model")
    for epoch in range(epochs):
        print("-"*25, f"Epoch {epoch + 1}","-"*25)
        
        train_loss = train_loop(model, opt, loss_fn, train_dataloader)
        train_loss_list += [train_loss]
        
        validation_loss = validation_loop(model, loss_fn, val_dataloader)
        validation_loss_list += [validation_loss]
        
        print(f"Training loss: {train_loss:.4f}")
        print(f"Validation loss: {validation_loss:.4f}")
        print()
        
    return train_loss_list, validation_loss_list
    
train_loss_list, validation_loss_list = fit(model, opt, loss_fn, train_dataloader, val_dataloader, 20)

Training and validating model
------------------------- Epoch 1 -------------------------
tensor([[[ 3.0000e+01,  3.0000e+01,  3.0000e+01,  ...,  3.0000e+01,
           3.0000e+01,  3.0000e+01],
         [ 1.5674e-02, -1.7720e+00,  1.5797e+00,  ...,  2.3214e-02,
          -1.8515e-01,  2.8794e-01],
         [ 5.0000e+01,  5.0000e+01,  5.0000e+01,  ...,  5.0000e+01,
           5.0000e+01,  5.0000e+01],
         ...,
         [-1.0000e+01, -1.0000e+01, -1.0000e+01,  ..., -1.0000e+01,
          -1.0000e+01, -1.0000e+01],
         [-1.0000e+01, -1.0000e+01, -1.0000e+01,  ..., -1.0000e+01,
          -1.0000e+01, -1.0000e+01],
         [-1.0000e+01, -1.0000e+01, -1.0000e+01,  ..., -1.0000e+01,
          -1.0000e+01, -1.0000e+01]],

        [[ 3.0000e+01,  3.0000e+01,  3.0000e+01,  ...,  3.0000e+01,
           3.0000e+01,  3.0000e+01],
         [ 5.8743e-01, -7.3378e-02,  1.8144e+00,  ..., -1.4157e+00,
           6.6397e-01,  1.1064e-01],
         [ 5.0000e+01,  5.0000e+01,  5.0000e+01,  ...,

C:\Users\okan2\AppData\Local\Temp\ipykernel_25320\4142664444.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X, y = torch.tensor(X).to(device), torch.tensor(y).to(device)


tensor([[[ 30.,  30.,  30.,  ...,   0.,   0.,   0.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [ 50.,  50.,  50.,  ...,   0.,   0.,   0.],
         ...,
         [-10., -10., -10.,  ...,   0.,   0.,   0.],
         [-10., -10., -10.,  ...,   0.,   0.,   0.],
         [-10., -10., -10.,  ...,   0.,   0.,   0.]],

        [[ 30.,  30.,  30.,  ...,   0.,   0.,   0.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [ 50.,  50.,  50.,  ...,   0.,   0.,   0.],
         ...,
         [-10., -10., -10.,  ...,   0.,   0.,   0.],
         [-10., -10., -10.,  ...,   0.,   0.,   0.],
         [-10., -10., -10.,  ...,   0.,   0.,   0.]],

        [[ 30.,  30.,  30.,  ...,   0.,   0.,   0.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [ 50.,  50.,  50.,  ...,   0.,   0.,   0.],
         ...,
         [-10., -10., -10.,  ...,   0.,   0.,   0.],
         [-10., -10., -10.,  ...,   0.,   0.,   0.],
         [-10., -10., -10.,  ...,   0.,   0.,   0.]],

c:\Users\okan2\anaconda3\envs\animationSVG\lib\site-packages\torch\nn\functional.py:5076: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


tensor([[[ 3.0000e+01,  3.0000e+01,  3.0000e+01,  ...,  3.0000e+01,
           3.0000e+01,  3.0000e+01],
         [ 8.7837e-01, -7.9115e-01,  1.0676e+00,  ..., -1.2390e-01,
           6.0168e-01,  5.3268e-01],
         [-2.9481e-02, -1.0977e+00,  8.8997e-01,  ...,  6.8793e-01,
          -3.0686e-01,  2.8863e-01],
         ...,
         [-1.0000e+01, -1.0000e+01, -1.0000e+01,  ..., -1.0000e+01,
          -1.0000e+01, -1.0000e+01],
         [-1.0000e+01, -1.0000e+01, -1.0000e+01,  ..., -1.0000e+01,
          -1.0000e+01, -1.0000e+01],
         [-1.0000e+01, -1.0000e+01, -1.0000e+01,  ..., -1.0000e+01,
          -1.0000e+01, -1.0000e+01]],

        [[ 3.0000e+01,  3.0000e+01,  3.0000e+01,  ...,  3.0000e+01,
           3.0000e+01,  3.0000e+01],
         [ 3.1958e-01, -1.0585e+00,  2.0784e+00,  ..., -6.4420e-01,
          -7.4325e-01,  8.1950e-01],
         [ 2.7401e-01, -4.5746e-01,  2.0082e+00,  ..., -1.1320e+00,
          -6.4636e-02,  7.0418e-01],
         ...,
         [-1.0000e+01, -1

C:\Users\okan2\AppData\Local\Temp\ipykernel_25320\2046223158.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X, y = torch.tensor(X, device=device), torch.tensor(y, device=device)


tensor([[[ 3.0000e+01,  3.0000e+01,  3.0000e+01,  ...,  3.0000e+01,
           3.0000e+01,  3.0000e+01],
         [ 9.4305e-01, -1.2392e+00,  7.8981e-01,  ...,  5.6730e-02,
          -4.7652e-01,  8.1883e-03],
         [ 5.0000e+01,  5.0000e+01,  5.0000e+01,  ...,  5.0000e+01,
           5.0000e+01,  5.0000e+01],
         ...,
         [-1.0000e+01, -1.0000e+01, -1.0000e+01,  ..., -1.0000e+01,
          -1.0000e+01, -1.0000e+01],
         [-1.0000e+01, -1.0000e+01, -1.0000e+01,  ..., -1.0000e+01,
          -1.0000e+01, -1.0000e+01],
         [-1.0000e+01, -1.0000e+01, -1.0000e+01,  ..., -1.0000e+01,
          -1.0000e+01, -1.0000e+01]],

        [[ 3.0000e+01,  3.0000e+01,  3.0000e+01,  ...,  3.0000e+01,
           3.0000e+01,  3.0000e+01],
         [ 4.2253e-01,  1.1574e+00,  1.2076e+00,  ..., -8.0198e-01,
           1.0202e+00,  1.8397e+00],
         [ 4.2253e-01,  1.1574e+00,  1.2076e+00,  ..., -8.0198e-01,
           1.0202e+00,  1.8397e+00],
         ...,
         [-1.0000e+01, -1

KeyboardInterrupt: 

In [15]:
test.shape

torch.Size([1, 14, 256])

In [58]:
def predict(model, input_sequence, max_length=maxLen, SOS_token=[[30] * 256], EOS_token=[[50] * 256]):
    """
    Method from "A detailed guide to Pytorch's nn.Transformer() module.", by
    Daniel Melchor: https://medium.com/@danielmelchor/a-detailed-guide-to-pytorchs-nn-transformer-module-c80afbc9ffb1
    """
    model.eval()
    
    y_input = torch.tensor([SOS_token], dtype=torch.long, device=device)
    
    num_tokens = len(input_sequence[0])

    for i in range(max_length):
        # Get source mask
        tgt_mask = model.get_tgt_mask(y_input.size(1)).to(device)
        
        pred = model(input_sequence, y_input, tgt_mask)
        
        next_item = pred.topk(1)[1] # num with highest probability
        #next_item = torch.tensor([[next_item]], device=device)
        #print(next_item, next_item.shape)

        next_item = pred[:,i-1:i,:]
        #print(next_item, next_item.shape)

        #print(input_sequence.shape, y_input.shape, next_item.shape, pred.shape)
        # Concatenate previous input with predicted best word
        y_input = torch.cat((y_input, next_item), dim=1)

        # Stop if model predicts end of sentence
        if next_item == EOS_token:
            break

    return y_input

In [60]:
result = predict(model, test)
print(result, result.shape)
model_parameters = result[:,1:2,-12:].squeeze().tolist()
model_parameters = [value - math.floor(value) for value in model_parameters]
model_parameters = [round(value) if index < 6 else value for index, value in enumerate(model_parameters)]
print(model_parameters)

model_parameters = pd.DataFrame({"animation_id" : 2, "model_output" : [model_parameters]})

print(model_parameters, model_parameters.shape)


tensor([[[ 30.0000,  30.0000,  30.0000,  ...,  30.0000,  30.0000,  30.0000],
         [-33.7552, -35.2907, -34.0070,  ...,  -0.2211,  -1.3127,   0.3068],
         [-33.8575, -35.3568, -34.0774,  ...,  -0.2684,  -1.3296,   0.3217],
         ...,
         [-33.8917, -35.3809, -34.1049,  ...,  -0.2784,  -1.3250,   0.3123],
         [-33.8927, -35.3819, -34.1059,  ...,  -0.2785,  -1.3251,   0.3118],
         [-33.8936, -35.3828, -34.1066,  ...,  -0.2786,  -1.3252,   0.3113]]],
       device='cuda:0', grad_fn=<CatBackward0>) torch.Size([1, 12, 256])
[1, 1, 1, 0, 0, 0, 0.22345733642578125, 0.02572190761566162, 0.7888292074203491, 0.7788923233747482, 0.6872642040252686, 0.30678504705429077]
   animation_id                                       model_output
0             2  [1, 1, 1, 0, 0, 0, 0.22345733642578125, 0.0257... (1, 2)


In [61]:
result = predict(model, test)

result = result.cpu()
model_parameters = result[:,1:,12:].squeeze(0).detach().numpy()
print(model_parameters, model_parameters.shape)

model_parameters = pd.DataFrame(model_parameters)
model_parameters["model_output"] = model_parameters.apply(lambda row: row.tolist(), axis=1)

def process_model_output(lst):
    # Floor all values in the list
    lst = [value - math.floor(value) for value in lst]
    
    # Round the first 6 values in the list
    lst[:6] = [round(value) for value in lst[:6]]
    
    return lst

# Apply the custom function to the "model_output" column
model_parameters = model_parameters[['model_output']]
model_parameters['model_output'] = model_parameters['model_output'].apply(process_model_output)

model_parameters["animation_id"] = range(1, len(model_parameters)+1)
print(model_parameters, model_parameters.shape)

[[-0.7928452  -0.5132692  -0.8413945  ... -0.22110768 -1.3127358
   0.30678505]
 [-0.81447136 -0.5080966  -0.8825949  ... -0.26839703 -1.3295602
   0.3217063 ]
 [-0.8202703  -0.5095601  -0.88736016 ... -0.27352238 -1.3266493
   0.31939757]
 ...
 [-0.82944894 -0.5123199  -0.8947391  ... -0.27840585 -1.3249863
   0.31230044]
 [-0.82998604 -0.512447   -0.89522606 ... -0.27850282 -1.3251035
   0.3117662 ]
 [-0.8304337  -0.51253235 -0.89563113 ... -0.27856076 -1.3252093
   0.31129926]] (11, 244)
                                         model_output  animation_id
0   [0, 0, 0, 0, 1, 0, 0.22417369484901428, 0.9911...             1
1   [0, 0, 0, 0, 1, 0, 0.23618638515472412, 0.9870...             2
2   [0, 0, 0, 0, 1, 0, 0.23963922262191772, 0.9842...             3
3   [0, 0, 0, 0, 1, 0, 0.24166133999824524, 0.9822...             4
4   [0, 0, 0, 0, 1, 0, 0.24295824766159058, 0.9806...             5
5   [0, 0, 0, 0, 1, 0, 0.24382394552230835, 0.9793...             6
6   [0, 0, 0, 0, 1, 0, 0.244

In [54]:
from src.postprocessing.postprocessing import *

postprocess_logo(model_parameters, "data/1_inserted_animation_id/logo_0.svg")

TypeError: cannot unpack non-iterable NoneType object